In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from PynPoint.core import Pypeline

In [3]:
pipeline = Pypeline("/Users/markusbonse/Desktop/",
                    "/Users/markusbonse/Desktop/",
                    "/Users/markusbonse/Desktop/RESULTS/HR8799")

Initiating PynPoint... [DONE]


## BP Clean

In [4]:
from PynPoint.processing_modules import BadPixelCleaningSigmaFilterModule

In [19]:
bp_cleaing = BadPixelCleaningSigmaFilterModule(name_in="bad_pixel_cleaning",
                                               image_in_tag="07_star_arr_normalized",
                                               image_out_tag="07_star_arr_normalized_bp_cleaned",
                                               box=9,
                                               sigma=4,
                                               iterate=1)
pipeline.add_module(bp_cleaing)

/Users/markusbonse/Documents/Arbeit/ETH/Git/PynPoint/PynPoint/core/Pypeline.py:228: UserWarning: Processing module names need to be unique. Overwriting the old Module
  warnings.warn('Processing module names need to be unique. Overwriting the old Module')


In [20]:
pipeline.run_module("bad_pixel_cleaning")

Validating module bad_pixel_cleaning... [DONE]
Running BadPixelCleaningSigmaFilterModule... [DONE]


## Stack the data

In [21]:
from PynPoint.processing_modules import StackAndSubsetModule

In [22]:
stacking = StackAndSubsetModule(name_in="stacking", 
                                image_in_tag="07_star_arr_normalized_bp_cleaned",
                                image_out_tag="07_star_arr_stacked",
                                random_subset=None,
                                stacking=40)
pipeline.add_module(stacking)

/Users/markusbonse/Documents/Arbeit/ETH/Git/PynPoint/PynPoint/core/Pypeline.py:228: UserWarning: Processing module names need to be unique. Overwriting the old Module
  warnings.warn('Processing module names need to be unique. Overwriting the old Module')


In [23]:
pipeline.run_module("stacking")

Validating module stacking... [DONE]


/Users/markusbonse/Documents/Arbeit/ETH/Git/PynPoint/PynPoint/core/DataIO.py:409: UserWarning: No attribute found - requested: NFRAMES
  warnings.warn('No attribute found - requested: %s' % name)


## Run PCAs for stacked version

In [24]:
# 1.) Prepare
from PynPoint.processing_modules import PSFdataPreparation

In [25]:
prepare = PSFdataPreparation(name_in="psf_prep1",
                             image_in_tag="07_star_arr_stacked",
                             image_out_tag="08_star_arr_stacked_prep",
                             image_mask_out_tag="not_needed",
                             F_final=1.0,
                             cent_size=0.07,
                             mask_out_tag="no_wv_mask_arr")
pipeline.add_module(prepare)

/Users/markusbonse/Documents/Arbeit/ETH/Git/PynPoint/PynPoint/core/Pypeline.py:228: UserWarning: Processing module names need to be unique. Overwriting the old Module
  warnings.warn('Processing module names need to be unique. Overwriting the old Module')


In [26]:
pipeline.run_module("psf_prep1")

Validating module psf_prep1... [DONE]


## Run the new PCA PSF subtraction

In [27]:
from PynPoint.processing_modules import FastPCAModule

In [28]:
pca = FastPCAModule(range(10),
                    name_in="PSF_subtraction1",
                    images_in_tag="08_star_arr_stacked_prep",
                    reference_in_tag="08_star_arr_stacked_prep",
                    res_mean_tag="res_mean_stacked")
pipeline.add_module(pca)

/Users/markusbonse/Documents/Arbeit/ETH/Git/PynPoint/PynPoint/core/Pypeline.py:228: UserWarning: Processing module names need to be unique. Overwriting the old Module
  warnings.warn('Processing module names need to be unique. Overwriting the old Module')


In [29]:
pipeline.run_module("PSF_subtraction1")

Validating module PSF_subtraction1... [DONE]
Calculating PSF model... [DONE]
Creating residuals... [DONE]


## Wirte out the results

In [30]:
from PynPoint.io_modules import FitsWritingModule

In [31]:
writeing = FitsWritingModule("mean_no_wv_stacked.fits",
                             name_in="fits_writing_stacked",
                             data_tag="res_mean_stacked")
pipeline.add_module(writeing)

/Users/markusbonse/Documents/Arbeit/ETH/Git/PynPoint/PynPoint/core/Pypeline.py:228: UserWarning: Processing module names need to be unique. Overwriting the old Module
  warnings.warn('Processing module names need to be unique. Overwriting the old Module')


In [32]:
pipeline.run_module("fits_writing_stacked")

Validating module fits_writing_stacked... [DONE]
Running FitsWritingModule... [DONE]


## Run the new PCA PSF subtraction with the non stacked data

In [16]:
prepare = PSFdataPreparation(name_in="psf_prep2",
                             image_in_tag="07_star_arr_normalized",
                             image_out_tag="08_star_arr_prep",
                             image_mask_out_tag="not_needed",
                             F_final=1.0,
                             cent_size=0.07,
                             mask_out_tag="no_wv_mask_arr")
pipeline.add_module(prepare)

/Users/markusbonse/Documents/Arbeit/ETH/Git/PynPoint/PynPoint/core/Pypeline.py:228: UserWarning: Processing module names need to be unique. Overwriting the old Module
  warnings.warn('Processing module names need to be unique. Overwriting the old Module')


In [ ]:
pipeline.run_module("psf_prep2")

In [17]:
pca = FastPCAModule(range(100),
                    name_in="PSF_subtraction2",
                    images_in_tag="08_star_arr_prep",
                    reference_in_tag="08_star_arr_prep",
                    res_mean_tag="res_mean")
pipeline.add_module(pca)

In [ ]:
pipeline.run_module("PSF_subtraction2")

In [18]:
writeing = FitsWritingModule("mean_no_wv.fits",
                             name_in="fits_writing",
                             data_tag="res_mean")
pipeline.add_module(writeing)

In [ ]:
pipeline.run_module("fits_writing")